<a href="https://colab.research.google.com/github/2002yoons/web-yunse/blob/main/AI_05_%EC%9D%B4%EC%9C%A4%EC%84%B8_Section1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EDA

데이터 디멘션 확인"

 ## 1.데이터 초기 확인

내용:
*   불러오기
*   피쳐확인
*   디멘션확인
*   결측치 확인 후 조치: 데이터가 정규분포가 따르지 않음(평균값으로 대체 불가) 유의미한 특성을 띄지 않으므로 *제거*
*   무의미한 컬럼 제거: ['Unnamed: 0'] , ['Publisher']



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#라이브러리 불러오기
import pandas as pd
import seaborn as sns
from scipy.stats import f_oneway
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib as mpl
mpl.rc("font", family='AppleGothic')

df_raw = pd.read_csv('/content/drive/MyDrive/game_data.csv')

In [ ]:
#피쳐 확인
df_raw.columns

In [ ]:
#디멘션 확인
df_raw.shape

In [ ]:
#결측치 확인
df_raw.isnull().sum()

In [ ]:
#결측치 확인 후 삭제
df_raw.dropna(inplace=True)
df_raw.drop(columns = ['Unnamed: 0','Publisher'],inplace = True)



In [ ]:
#변수 생성
df = df_raw
df_=df_raw


##  2. 데이터 2차 확인

내용:
*   출고량 피쳐, float으로 전환
*   년도 통일
*   '전지역 출고량' 피쳐 추가
*   데이터셋 경량화: 전지역 출고량 100미만인 데이터는 무의미하다고 판단되어 제거 후 따로 저장

 




In [ ]:
# 출고량 데이터 타입을 숫자형으로 반환:
def value_to_float(x):
    if 'K' in x:     
        return (float(x.replace('K','')))*1000
    if 'M' in x:  
        return (float(x.replace('M','')))*1000000
    else: 
        return float(x)

df['NA_Sales'] = df['NA_Sales'].apply(value_to_float)
df['EU_Sales'] = df['EU_Sales'].apply(value_to_float)
df['JP_Sales'] = df['JP_Sales'].apply(value_to_float)
df['Other_Sales'] = df['Other_Sales'].apply(value_to_float)


In [ ]:
# 년도 바꾸는 함수 작성 후 적용
def year_upgrade(x):
    x = int(x)
    x = str(x)
    if len(x)==1 :
        return int('200'+x)
    if len(x)<3 and int(x)<21:
        return int('20'+x)
    if len(x)<3 and int(x)>21:
        return int('19'+x)
    else:
        return x

df['Year']= df['Year'].apply(year_upgrade).astype(int)



In [ ]:
#총 출고량 피쳐 추가
df['Whole_Sales'] = df.iloc[:,4:8].sum(axis=1)
df

In [ ]:
# 게임 중 전체지역 출고량 추가
# 전체 출고량 100 미달인 데이터 삭제 후 df_simple에 저장
df['Whole_Sales'] = df.iloc[:,4:8].sum(axis =1)
filt=(df['Whole_Sales']<1000)
df_simple = df.loc[~filt].sort_values(by='Whole_Sales').reset_index(drop = True)
df_simple = pd.DataFrame(data = df_simple)

In [ ]:
df_simple

# 데이터 분석

## 1.지역에 따라 선호하는 게임 장르가 달랐을까?


In [ ]:
#데이터 셋 결합 index: 장르, columns = 지역
df_region_genre = df[['Genre','NA_Sales', 'EU_Sales', 'JP_Sales', 'Other_Sales']]
df_region_genre = df_region_genre.groupby(by = 'Genre',as_index = True).sum().astype('category')

#카이검정 
chi2 = chi2_contingency(df_region_genre)
chi2[1]

In [ ]:
#p값이 0이므로,지역에 따라 선호하는 게임 장르는 같다고 보기 힘듭니다.

##  2.2010년도 부터 2020년동안 지역에 따라 선호되는 장르는 달랐을까?

In [ ]:
#2010년도 이후 장르별 출고량
filt_2010 = (df_simple['Year']>=2010)
df_2010 = df_simple.loc[filt_2010].groupby(by = 'Genre').sum().iloc[:,1:-1]
#카이검정
chi2_2010 = chi2_contingency(df_2010)
chi2_2010

##  3.년도별 또는 세대별로 게임 트렌드가 다를까?






### 3-a 년도별 총 출고량 그래프 

In [ ]:
#장르에 따른 년도별 총 출고량 그래프 
sns.set_theme(style="darkgrid")
plt.figure(figsize=(22, 7))
sns.lineplot(data=df_simple,x='Year',y='Whole_Sales',hue= 'Genre')


### 3-a. 년도별 게임 트렌드 분석

In [ ]:
#년도와 장르별로 카이검정 실행
a = df_simple[['Year','Genre']].astype('category')
obs =pd.crosstab(a['Year'],a['Genre'])
chi2_contingency(obs)[1]

### 3-b. 세대별 게임 트렌드 분석

In [ ]:
# 년도와 장르, 총 출고량 데이터 조합
df_Genre = df_simple[['Year','Genre','Whole_Sales']]
#년도 피쳐를 10년 단위로 카테고리화 후 '세대' 피쳐 생성
category = pd.cut(df_Genre['Year'],bins=[1980,1990,2000,2010,2020],labels=['80년대','90년대','00년대','10년대'])
df_Genre.insert(2,'세대', category)
#세대와 장르 데이터 카테고리 타입 변환 후 크로스탭 생성
df_Genre[['세대','Genre']].astype('category')
obs_year_genre = pd.crosstab(df_Genre['세대'],df_Genre['Genre'])
obs_year_genre
#카이검정
chi2_year_genre = chi2_contingency(obs_year_genre)
chi2_year_genre[1]

결론: 게임 장르는 지역 또는 세대와 연관성이 없다. 


## 4.지역 별 출고량이 높은 게임의 분석 및 시각화

### 4-a 1980년대 이후 전지역 장르별 출고량 비율


In [ ]:
#
df_wholeSales = df.groupby(by = 'Genre',as_index=True).sum().iloc[:,-1:].astype(float)
df_wholeSales2 = df_simple.loc[filt_2010].groupby(by = 'Genre',as_index=True).sum().iloc[:,-1:].astype(float)
df_wholeSales3 = df_simple.loc[~filt_2010].groupby(by = 'Genre',as_index=True).sum().iloc[:,-1:].astype(float)
df_wholeSales12 =pd.concat([df_wholeSales3,df_wholeSales2],axis=1,)
df_wholeSales12.plot.pie(subplots=True, figsize=(18, 14),autopct='%.2f%%',)
#좌: 1980년도부터 2010년도 전까지 장르별 총 출고량 비율///우: 2010년도부터 현재까지 총 출고량 비율
#결론: 액션과 슈팅 장르의 비중이 점점 커짐.

### 4-a 지역별 총 출고량



In [ ]:
# 지역별 총 출고량 
df_Sales=df_.iloc[:,4:8].sum(axis=0).reset_index()
df_Sales.columns = ['region','Sales']
sns.barplot(data=df_Sales,x='region',y='Sales')

### 4-b 북미지역 출고량 TOP 10 

In [ ]:
#북미지역 출고량 top 10 장르별 그래픽

#데이터셋 조합
NA_ = df_simple.sort_values(by ='NA_Sales',ascending = False)
NA_top10 = NA_.reset_index(drop =True).iloc[0:10,0:5]
#데이터셋 장르별 카테고리화
NA_Genre = NA_top10.groupby('Genre')['NA_Sales'].sum().reset_index().sort_values(by ='NA_Sales',ascending =False)
NA_Genre
#그래프 작성
fig = plt.figure(figsize=(25, 10))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)


plt.subplot(212)
plt.title('North America')
sns.barplot(x="Genre", y="NA_Sales", data=NA_Genre)
plt.xticks(fontsize=16)
plt.yticks(fontsize=10)
plt.grid

plt.subplot(211)
plt.title('North America')
sns.barplot(x="Name", y="NA_Sales", data=NA_top10,hue='Genre')
plt.xticks(fontsize=10, rotation=20)
plt.grid

### 4-c 유럽지역 출고량 TOP 10 

In [ ]:
#유럽지역 출고량 top 10 장르별 그래픽
EU_ = df_simple.sort_values(by ='EU_Sales',ascending = False)
EU_top10 = EU_.reset_index(drop =True).iloc[0:10,0:6].drop(columns = 'NA_Sales')

EU_Genre = EU_top10.groupby('Genre')['EU_Sales'].sum().reset_index().sort_values(by ='EU_Sales',ascending =False)


fig = plt.figure(figsize=(25, 10))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)


plt.subplot(212)
sns.barplot(x="Genre", y="EU_Sales", data=EU_)
plt.xticks(fontsize=16)
plt.yticks(fontsize=10)

plt.grid

plt.subplot(211)
sns.barplot(x="Name", y="EU_Sales", data=EU_top10,hue='Genre')
plt.xticks(fontsize=10, rotation=20)
plt.grid

### 4-d 일본지역 출고량 TOP 10 

In [ ]:
#일본지역 출고량 top 10 장르별 그래픽
JP_ = df_simple.sort_values(by ='JP_Sales',ascending = False)
JP_top10 = JP_.reset_index(drop =True).iloc[0:10,0:7].drop(columns=['NA_Sales','EU_Sales'])


JP_Genre = JP_top10.groupby('Genre')['JP_Sales'].sum().reset_index().sort_values(by ='JP_Sales',ascending =False)


fig = plt.figure(figsize=(25, 10))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)


plt.subplot(212)
sns.barplot(x="Genre", y="JP_Sales", data=JP_)
plt.xticks(fontsize=16)
plt.yticks(fontsize=10)
plt.grid

plt.subplot(211)
sns.barplot(x="Name", y="JP_Sales", data=JP_top10,hue='Genre')
plt.xticks(fontsize=10, rotation=20)
plt.grid

### 4-e 기타지역 출고량 TOP 10 

In [ ]:
Oth_ = df_simple.sort_values(by ='Other_Sales',ascending = False)
Oth_top10 = Oth_.reset_index(drop =True).iloc[0:10,0:8].drop(columns= ['NA_Sales','EU_Sales','JP_Sales'])

Oth_Genre = Oth_top10.groupby('Genre')['Other_Sales'].sum().reset_index().sort_values(by ='Other_Sales',ascending =False)


fig = plt.figure(figsize=(25, 10))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)


plt.subplot(212)
sns.barplot(x="Genre", y="Other_Sales", data=Oth_)
plt.xticks(fontsize=16)
plt.yticks(fontsize=10)
plt.grid

plt.subplot(211)
sns.barplot(x="Name", y="Other_Sales", data=Oth_top10,hue='Genre')
plt.xticks(fontsize=10, rotation=20)
plt.grid



In [ ]:
#그렇다면, 과연 과거의 데이터가 현재 게임 설계에 도움이 될까?


## 5. 기존 IP를 이용하는 건 어떨까?

In [ ]:
#포켓몬스터 IP 게임장르
df_pok = df[df_['Name'].str.match('Pok')]
a = df_pok['Genre'].value_counts().reset_index()
a.columns =['Genre','counts']
a
#총 49개 결과

In [ ]:
#마리오 IP 게임장르
df_mario = df[df_['Name'].str.match('Mario')]
b = df_mario['Genre'].value_counts().reset_index()
b.columns =['Genre','counts']
#총 64개


In [ ]:
#마리오 포켓몬 장르 별 차트 

fig = plt.figure(figsize=(18, 10))
ax1 = fig.add_subplot(2, 2, 1)
ax2 = fig.add_subplot(2, 2, 2)
ax3 = fig.add_subplot(2, 2, 3)
ax4 = fig.add_subplot(2, 2, 4)

#포켓몬 IP 별 장르수 그래프
plt.subplot(221)
sns.barplot(data=a, x = 'Genre',y='counts')
plt.title('pokemon IP')
plt.xticks(fontsize=10, rotation=20)
plt.grid
#마리오 IP 별 장르수 그래프
plt.subplot(222)
sns.barplot(data=b, x = 'Genre',y='counts')
plt.xticks(fontsize=10, rotation=20)
plt.title('Mario IP')
plt.grid
#포켓몬 IP 장르별 출고량 그래프
plt.subplot(223)
sns.barplot(data= df_pok, x = 'Genre',y='Whole_Sales')
plt.xticks(fontsize=10, rotation=20)
plt.grid
#마리오 IP 장르별 출고량 그래프
plt.subplot(224)
sns.barplot(data= df_mario, x = 'Genre',y='Whole_Sales')
plt.xticks(fontsize=10, rotation=20)
plt.grid


#결론:

분석 결과:

1.지역에 따른 게임 장르가 다르므로 집중적인 공략이 필요

2.북미지역 출고량 최다, 액션 장르가 대세.

3.세대별로 게임 트렌드가 다르기에 2021년인 지금 다른 장르가 핫해질 가능성 유

4 IP를 연장시키기엔 리스크가 큼

5.클라우드 게임의 선두주자는 마이크로소프트, 즉 xbox 플랫폼이 전략적으로 
유리함.

결론: 새로운 ip로 북미지역을 타게팅한 액션장르 게임을 xbox에 출시.



